### Connect to server

In [2]:
import iris
conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO") # Server, Port , Namespace, Username, Password
cursor = conn.cursor()

### Make database

In [1]:
print("Connected to InterSystems IRIS")

Connected to InterSystems IRIS


In [3]:
import pandas as pd


In [4]:
#Create database
df = pd.read_json("./data/data.json") #pd.DataFrame(out, columns=cols)  #replace with Tristans/Iaroslavs code




### Make encoding / add it to database


In [ ]:
#Make encoding 
#pip install sentence-transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') #might not work, if this is the case use instead 'all-MiniLM-L6-v2'

embeddings = model.encode(df['content'].tolist(), normalize_embeddings=True, show_progress_bar=True)


df['vector'] = embeddings.tolist()

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [47]:
###Add to Iris:

table_name = "VectorSearch.ORGstruct"

create_table_query = f"""
CREATE TABLE {table_name} (
id INTEGER,
filename LONGVARCHAR,
content LONGVARCHAR,
vector VECTOR(DOUBLE, 384)
)
"""

cursor.execute(f"DROP TABLE IF EXISTS {table_name}" )
cursor.execute(create_table_query)



insert_query = f"INSERT INTO {table_name} (id, filename, content, vector) values (?, ?, ?, TO_VECTOR(?))"
df["vector"] = df["vector"].astype(str)

rows_list = df[["id", "filename", "content", "vector"]].values.tolist()
cursor.executemany(insert_query, rows_list)


("Batch Row 1 [ SQLCODE: <-104>:<Field validation failed in INSERT, or value failed to convert in DisplayToLogical or OdbcToLogical> <Field 'VectorSearch.ORGstruct.vector' (value 'D4B2E9A984CBE89CF936D4202C479010@$vector') failed validation>]",
 "Batch Row 2 [ SQLCODE: <-104>:<Field validation failed in INSERT, or value failed to convert in DisplayToLogical or OdbcToLogical> <Field 'VectorSearch.ORGstruct.vector' (value '9F4B651891489E2628B3A7698BE57FE9@$vector') failed validation>]",
 "Batch Row 3 [ SQLCODE: <-104>:<Field validation failed in INSERT, or value failed to convert in DisplayToLogical or OdbcToLogical> <Field 'VectorSearch.ORGstruct.vector' (value '2F2379139D0EA8E1A31799789814F9F2@$vector') failed validation>]",
 "Batch Row 4 [ SQLCODE: <-104>:<Field validation failed in INSERT, or value failed to convert in DisplayToLogical or OdbcToLogical> <Field 'VectorSearch.ORGstruct.vector' (value '64808E9A7B99FE65835EE72F530A110A@$vector') failed validation>]",
 "Batch Row 5 [ SQLC

### LLM setup (Ollama)

In [ ]:
#pip install ollama
#pip install langchain langchain-ollama 


from ollama import chat
from ollama import ChatResponse
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama


In [ ]:
import iris
class RAGChatbot:
    def __init__(self):
        self.message_count = 0
        conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO") # Server, Port , Namespace, Username, Password
        self.cursor = conn.cursor()
        self.agent = self.create_agent()
        self.embedding_model = self.get_embedding_model()
        
        
    def get_embedding_model(self):
        return  SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
        
    def create_agent(self):
        # Initialize model
        llm = ChatOllama(model="gemma3:1b") 
        
        # Initialise short-term memory
        checkpointer = InMemorySaver()
        
        # Create model
        agent = create_agent(
            model=llm, # Set model as our LLM 
            middleware=[
                # create summarization proceedure - this creates summaries of our conversation to keep memory brief.
                SummarizationMiddleware(
                    model=llm,
                    max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
                    messages_to_keep=20,  # Keep last 20 messages after summary
                )
            ],
            # Creates the agent's memory with pre-initialized model
            checkpointer=checkpointer,
        )
        self.config = {"configurable": {"thread_id": "1"}}
        return agent
        
    def vector_search(self, user_prompt): ###Bude potreba hodne upravit
        search_vector =  self.embedding_model.encode(user_prompt, normalize_embeddings=False, show_progress_bar=False).tolist() 
        
        search_sql = f"""
            SELECT TOP 5 content 
            FROM VectorSearch.ORGstruct
            ORDER BY VECTOR_DISTANCE(vector, TO_VECTOR(?,double)) DESC
        """
        self.cursor.execute(search_sql,[str(search_vector)])
        
        results = self.cursor.fetchall()
        return results

    def get_prompt(self):
       
        query = input("\n\nHi, I'm a chatbot used for searching a patient's medical history. How can I help you today? \n\n - User: ")
    
        return query
    
    def validation(self, result):

        return result
    

    def return_response(self):
        query = self.get_prompt()

        search = True
        if self.message_count != 0:
            search_ans = input("Search the database? [Y/N - default N]")
            if search_ans.lower() != "y":
                search = False


        if search:
            try:
                patient_id = int(input("What is the patient ID?"))
            except:
                print("The patient ID should be an integer")
                return

            results = self.vector_search(query, patient_id)
            if results == []:
                print("No results found, check patient ID")
                return

            prompt = f"CONTEXT:\n{results}\n\nUSER QUESTION:\n{query}"
        else:
            prompt = f"USER QUESTION:\n{query}"

        ##print(prompt)
        system_prompt = "You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history using retrieved information from a database.\
        Please provide a detailed and medically relevant explanation, \
        include the dates of the information you are given."
        response = self.agent.invoke({"messages" : [("system", system_prompt), ("user", query), ("system", str(results))]}, self.config)
        
        response["messages"][-1].pretty_print()
        self.message_count += 1
        
        
        validated_response = self.validation(response)

        return validated_response 



### Interface with chatbot

In [ ]:
bot = RAGChatbot()
bot.run()